In [3]:
import glob
import json
import pandas as pd
import librosa
import os
import re
import numpy as np

In [4]:
#read files in an check ouptut
files = glob.glob('/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/' + "*.json")
files

['/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_21.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_01.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_17.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_16.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_20.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_07.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_11.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_27.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_26.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_10.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_06.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP/MP_13.json',
 '/Users/adj10085/Documents/data_editing/ELIC/ckm/MP

In [5]:
## this loop reads in the json file information and puts it into a dataframe 
file_name_list_all = []
text_all = []
xmax_all = []

for file in files:
    # get correct json file
    with open(file, "r") as f:
        data = json.load(f)
        
    file_name_curr = [x['audio'] for x in data]
    text_curr = [x['text'] for x in data]
    
    
    file_name_list_all = file_name_list_all + file_name_curr
    text_all = text_all + text_curr
    # get the list of file names and text

df = pd.DataFrame()
df['file_name'] = file_name_list_all
df['transcription'] = text_all

In [6]:
#this loop uses the dataframe to measure the duration of the MP3 file correpsonding to every row of the dataframe
#it then appends this info to a new column of the df
for index, row in df.iterrows():
    filename = df["file_name"].iloc[index]
    filename, extension  = filename.split(".mp3")
    filename = re.sub(pattern= "\.", repl= "_" , string = filename)
    audio, sr = librosa.load("/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP_mp3s.mp3/" + filename + ".mp3")
    xmax = librosa.get_duration(y = audio, sr=sr)
    xmax_all.append(xmax)
df['xmax'] = xmax_all


In [7]:
#double checking
df.head(10)

,file_name,transcription,xmax
0,MP_21_50.613-68.43.mp3,Tȃ put se pojavila lisica:\n• Bog daj!\n• Bog ...,17.817460
1,MP_21_69.12-86.32.mp3,"O, oprosti!\n• Ča bi to reć „pitoma“?\n• Ti ni...",17.200000
2,MP_21_87.07-105.52.mp3,I to je se ča jin se pjaža. Ča i ti išćeš koko...,18.450023
3,MP_21_106.31-123.91.mp3,Ti si za me leh mići mali ko stotine hiljad dr...,17.600000
4,MP_21_124.13-138.72.mp3,Ja za te jedina na svitu.\n• Počinjen razumit....,14.590023
5,MP_21_139.27-156.85.mp3,Bi rekli da je lisica bila sakrabojska kurijož...,17.580000
6,MP_21_157.79-174.76.mp3,"Lovin kokoše, a ljudi mene love. Sȅ kokoše sli...",16.970023
7,MP_21_175.03-193.57.mp3,"Tvoji će me, kot muzika, potezat van zi škulje...",18.540000
8,MP_21_194.16-209.67.mp3,Lipo će bit kad me udelaš pitomu! Zlatna šenic...,15.510023
9,MP_21_211.1-225.91.mp3,"Prosin te udelaj me pitomu!\n• Ću rad, ma nima...",14.810023


In [12]:
#this loop takes the df information row by row and creates a textgrid for every transcription section.
#it does not concatenate them into a larger textgrid
xmin = 0
tail = []
    
for index, row in df.iterrows():
    xmax = df["xmax"].iloc[index]
    text = df["transcription"].iloc[index]
    name = df["file_name"].iloc[index]
    filename, extension  = name.split(".mp3")
    filename = re.sub(pattern= "\.", repl= "_" , string = filename)
    
    lines_head = ['File type = "ooTextFile"',
    'Object class = "TextGrid"', '']
    
    tier_info = [
        'xmin = ' + str(xmin),
        'xmax = ' + str(xmax),
        'tiers? <exists>',
        'size = 2',
        'item []:',
        '    item [1]:',
        '        class = "IntervalTier"', 
        '        name = "null"',
        '        xmin = ' + str(xmin),
        '        xmax = ' + str(xmax),
        '        intervals: size = 1',
        '        intervals [1]:',
        '            xmin = ' + str(xmin),
        '            xmax = ' + str(xmax),
        '            text = ""', 
        '    item [2]:',
        '        class = "IntervalTier"',
        '        name = "MP"',
        '        xmin = ' + str(xmin),
        '        xmax = ' + str(xmax),
        '        intervals: size = 1',
        '        intervals [1]:',
        '            xmin = ' + str(xmin), 
        '            xmax = ' + str(xmax), 
        '            text = "' + str(text) + '"'
    ]
        
    lines = lines_head + tier_info
        
    with open("/Users/adj10085/Documents/data_editing/ELIC/ckm/MP_data.json/MP_textgrids/" + filename + '.TextGrid', 'w') as f:
        for line in lines:
            f.write("{}\n".format(line))
